# Transfer Learning Hubble Data

In [ ]:
%matplotlib notebook
from matplotlib.pyplot import *

from transfer_learning.fingerprint import FingerprintResnet
from transfer_learning.data_processing import CropData, GrayScaleData
from transfer_learning.cutouts import FullImageCutout
from transfer_learning.similarity import tSNE, Jaccard, Distance
from transfer_learning.transfer_learning import TransferLearning
from transfer_learning.transfer_learning_display import TransferLearningDisplay
import pickle

# Full width 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Load Data and Meta Data

First we need to create/load the information corresponding to the images.  The pickle file contains a list of dictionaries. Each dictionary corresponds to a hubble image and has the location (in this case hosted off an AWS site), RA/DEC and meta information. The meta information comes directly from astroquery.mast data.

In [ ]:
processing_dict = pickle.load(open('../data/hubble_acs.pck', 'rb'))
print(processing_dict[100])

## Fingerprint Creator

We need to create a fingerprint creator / classifier.  This method takes an image and calculates the corresponding weighted predictions based on the ImageNet data and a pre-trained network.  In this case we are going to use the ResNet50 pre-trained network but we could also use the VGG16, VGG19 etc. 

In [ ]:
fingerprint_model = FingerprintResnet()

## Image Pre-Processing

The image pre-processing model will be passed in to the transfer learning code and will be applied to each image before the classification/fingerprint is calculated.  In this case, we are going to use the full image (cropped to 224 x 224 as required by ImageNet) and then make sure it is formatted as gray scale (which all the images should be in this case).

In [ ]:
basic_cutout = FullImageCutout(output_size=224)

# Added Gray Scale as some were 3 channel gray scale
data_processing = [
            [CropData(), GrayScaleData()],
        ]

   ## Calculate the Fingerprints

Now we finally get to the part where the fingerprints are calculated. The transfer learning module takes the cutout, data processing and fingerprint modules defined above and then calculate_stream is called on the first 100 images (for example).

In [ ]:
tl = TransferLearning(basic_cutout, data_processing, fingerprint_model)
tl.calculate_stream(processing_dict[:100])

## Similarity Calculator

Before we display the similarity between the images we need to define how we are going to calculate the similarities. In this case we are going to use the tSNE data reduction method.  Though we could also use a set similarity method (Jaccard) or distance metric (Distance).

In [ ]:
similarity = tSNE(display_type='hexbin')

## Calculate Similarity

Next we need to calculate create the display code based on the type of simiilarity that was set above.

In [ ]:
tld = TransferLearningDisplay(similarity)

## Transfer Learning Simiarlity

Once you show the transfer learning display (next command) you will see a tSNE plot in the lower left, 9 images on the right and an Aitoff plot on the top left.  Click somewhere in the tSNE plot and it will display the corresponding image in the top left of the 9 images and show you the 8 corresponding similar images. You can hover over one of the 9 images to see the meta information about it along with the top 8 corresponding ImageNet image classifications.

In [ ]:
tld.show(tl.fingerprints)